In [4]:
%load_ext autoreload
%autoreload 2
from napari.qt.threading import thread_worker
import control
import napari
import gui
import time
import numpy as np
import toml
import threading

s = """
[camera]
device = "demo_camera"
[stage]
device = "demo_stage"
[valves]
device = "demo_valves"
"""

conf = toml.loads(s)
c = control.load(conf, path="/usr/local/lib/micro-manager")
c._core.setTimeoutMs(100000)
c.exposure = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
c.devices[0].px_len

6.5

In [3]:
gui.live(c)

In [4]:
e = threading.Event()
e.is_set()

False

In [49]:
def acq_func(g, xs, ys):
    s = 0
    print(xs, ys)
    xp = g.new_array("main", channel=["gfp", "cy5"], time=5, row=len(ys), col=len(xs))
    for j, t in enumerate(range(5)):
        xp.coords["bla"] = f"bloo{j}"
        for k, y in enumerate(ys):
            for l, x in enumerate(xs):
                c.xy = (x, y)
                for i, chan in enumerate(["gfp", "cy5"]):
                    # c.filter = chan
                    print(i, j, k, l)
                    time.sleep(1)
                    s += 1
                    xp[i, j, k, l] = np.ones(xp[i, j, k, l].shape) * s
                    yield

t = gui.acquire(c, "test", acq_func, overlap=0.1)

[5993.01 8989.51] [5993.01 8989.51]
0 0 0 0
1 0 0 0
0 0 0 1
1 0 0 1
0 0 1 0
1 0 1 0
0 0 1 1
1 0 1 1
0 1 0 0
1 1 0 0
0 1 0 1
1 1 0 1
0 1 1 0
1 1 1 0
0 1 1 1
1 1 1 1
0 2 0 0
1 2 0 0
0 2 0 1
1 2 0 1
0 2 1 0
1 2 1 0
0 2 1 1
1 2 1 1
0 3 0 0
1 3 0 0
0 3 0 1
1 3 0 1
0 3 1 0
1 3 1 0
0 3 1 1
1 3 1 1
0 4 0 0
1 4 0 0
0 4 0 1
1 4 0 1
0 4 1 0
1 4 1 0
0 4 1 1
1 4 1 1


In [50]:
c.x += 1000
c.wait()
c.y += 1000

In [20]:
t.

In [15]:
t.new_array("arr2", channel=["gfp", "cy5", "yfp"], time=7, var=3)

<xarray.DataArray 'arr2' (channel: 3, time: 7, var: 3, y: 512, x: 512)> Size: 17MB
dask.array<from-zarr, shape=(3, 7, 3, 512, 512), dtype=uint8, chunksize=(1, 1, 1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * channel  (channel) <U3 36B 'gfp' 'cy5' 'yfp'
Dimensions without coordinates: time, var, y, x
Attributes:
    overlap:   0.1
    acq_func:  def acq_func(g, xs, ys):\n    s = 0\n    print(xs, ys)\n    xp...

In [80]:
mg.image()(t.arrays["main"])

<xarray.Dataset> Size: 10MB
Dimensions:  (channel: 2, time: 5, tile_row: 2, tile_col: 2, tile_y: 512,
              tile_x: 512)
Coordinates:
  * channel  (channel) <U3 24B 'gfp' 'cy5'
    bla      <U5 20B 'bloo4'
Dimensions without coordinates: time, tile_row, tile_col, tile_y, tile_x
Data variables:
    tile     (channel, time, tile_row, tile_col, tile_y, tile_x) uint8 10MB dask.array<chunksize=(1, 1, 1, 1, 512, 512), meta=np.ndarray>
Attributes:
    overlap:   0.1
    acq_func:  def acq_func(g, xs, ys):\n    s = 0\n    print(xs, ys)\n    xp...


<xarray.Dataset> Size: 7MB
Dimensions:  (channel: 2, time: 5, im_y: 820, im_x: 820)
Coordinates:
  * channel  (channel) <U3 24B 'gfp' 'cy5'
    bla      <U5 20B 'bloo4'
Dimensions without coordinates: time, im_y, im_x
Data variables:
    image    (channel, time, im_y, im_x) uint8 7MB dask.array<chunksize=(1, 1, 410, 410), meta=np.ndarray>
Attributes:
    overlap:   0.1
    acq_func:  def acq_func(g, xs, ys):\n    s = 0\n    print(xs, ys)\n    xp...

In [82]:
mg.image()("*/main")

<xarray.Dataset> Size: 42MB
Dimensions:  (channel: 2, time: 5, tile_row: 2, tile_col: 2, tile_y: 512,
              tile_x: 512)
Coordinates:
  * channel  (channel) <U3 24B 'gfp' 'cy5'
Dimensions without coordinates: time, tile_row, tile_col, tile_y, tile_x
Data variables:
    bla      <U5 20B ...
    tile     (channel, time, tile_row, tile_col, tile_y, tile_x) float32 42MB dask.array<chunksize=(1, 1, 1, 1, 512, 512), meta=np.ndarray>
Attributes:
    acq_func:  def acq_func(g, xs, ys):\n    s = 0\n    print(xs, ys)\n    xp...
    overlap:   0.1


<xarray.Dataset> Size: 27MB
Dimensions:  (channel: 2, time: 5, im_y: 820, im_x: 820)
Coordinates:
  * channel  (channel) <U3 24B 'gfp' 'cy5'
Dimensions without coordinates: time, im_y, im_x
Data variables:
    bla      <U5 20B ...
    image    (channel, time, im_y, im_x) float32 27MB dask.array<chunksize=(1, 1, 410, 410), meta=np.ndarray>
Attributes:
    acq_func:  def acq_func(g, xs, ys):\n    s = 0\n    print(xs, ys)\n    xp...
    overlap:   0.1

In [65]:
import magnify as mg

In [43]:
isinstance(v[0].add_image(np.zeros((3, 10, 10))))

TypeError: object of type 'Image' has no len()

In [35]:
v.add_image(np.ones([5, 5, 5, 5, 5, 5, 5, 5]))

<Image layer 'Image [1]' at 0x79e3ecc6c910>

In [45]:
def a(*args):
    print(args)

a(1, 2, *(1, 2, 3), 4, 5)

(1, 2, 1, 2, 3, 4, 5)


[autoreload of gui failed: Traceback (most recent call last):
  File "/home/karlk/.bin/miniforge3/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/karlk/.bin/miniforge3/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/karlk/.bin/miniforge3/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/karlk/repos/prismo/src/prismo/gui.py", line 244
    img = img.transpose("channel", *self._viewer.dims.axis_labels, "

In [36]:
v.window._qt_viewer._controls.widgets[v.layers[0]].autoScaleBar._auto_btn.click()

In [45]:
v[0].dims.current_step

(0, 4, 4)

In [6]:
threading.enumerate()

[<_MainThread(MainThread, started 132158203393856)>,
 <Thread(IOPub, started daemon 132158125893184)>,
 <Heartbeat(Heartbeat, started daemon 132158117500480)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 132157888525888)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 132157880133184)>,
 <ControlThread(Control, started daemon 132157871740480)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 132157863347776)>,
 <ParentPollerUnix(Thread-2, started daemon 132157044844096)>]

In [14]:
threading.enumerate()

[<_MainThread(MainThread, started 133236044224320)>,
 <Thread(IOPub, started daemon 133235966711360)>,
 <Heartbeat(Heartbeat, started daemon 133235958318656)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 133235723990592)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 133235715597888)>,
 <ControlThread(Control, started daemon 133235707205184)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 133235698812480)>,
 <ParentPollerUnix(Thread-2, started daemon 133234885609024)>,
 <Thread(Thread-16 (run_worker), started 133233433830976)>]